# Custom Chatbot Project

TODO: In this cell, write an explanation of which dataset you have chosen and why it is appropriate for this task <br>
**Answer** : I have chosen to download the WIKIPEDIA news titles for 2023 - 2025 since the CHATGPT 3.5 turbo has a cutoff in 2022. 


## Data Wrangling

TODO: In the cells below, load your chosen dataset into a `pandas` dataframe with a column named `"text"`. This column should contain all of your text data, separated into at least 20 rows.

In [51]:
import openai

print(openai.__version__)
openai.api_base = "https://openai.vocareum.com/v1"
openai.api_key = "voc-1253443520126677434845468b589b4f03ad2.79010116"

0.28.0


In [52]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from dateutil.parser import parse

# Get the Wikipedia page for "2022" since OpenAI's models stop in 2021
titles = [2022, 2023, 2024, 2025, 'iPhone', 'List_of_large_language_models']

all_texts = []
for title in titles:
    params = {
        "action": "query",
        "format": "json",
        "prop": "extracts",
        "titles": str(title),
        "explaintext": True,
    }
    headers = {"User-Agent": "Prof.Cristian (https://udacity.com; contact@me.com)"}
    resp = requests.get(
        "https://en.wikipedia.org/w/api.php", params=params, headers=headers
    )
    page = next(iter(resp.json()["query"]["pages"].values()))
    title = page.get("title", "")
    text = page.get("extract", "")
    for paragraph in text.split("\n"):
        if paragraph.strip():
            all_texts.append({"title": title, "text": paragraph})

In [53]:
df = pd.DataFrame(all_texts)
display(df)
# Printăm frumos JSON-ul
# print(json.dumps(page, indent=2))
# print(resp.json())

,title,text
0,2022,2022 (MMXXII) was a common year starting on Sa...
1,2022,The year began with another wave in the COVID-...
2,2022,2022 was also dominated by wars and armed conf...
3,2022,== Conflicts ==
4,2022,The ongoing Russian invasion of Ukraine escala...
...,...,...
1163,List of large language models,== See also ==
1164,List of large language models,List of chatbots
1165,List of large language models,List of language model benchmarks
1166,List of large language models,== Notes ==


In [54]:
# Clean up text to remove empty lines and headings
df = df[(df["text"].str.len() > 0) & (~df["text"].str.startswith("=="))]

# In some cases dates are used as headings instead of being part of the
# text sample; adjust so dated text samples start with dates
prefix = ""
for i, row in df.iterrows():
    # If the row already has " - ", it already has the needed date prefix
    if " – " not in row["text"]:
        try:
            # If the row's text is a date, set it as the new prefix
            parse(row["text"])
            prefix = row["text"]
        except:
            # If the row's text isn't a date, add the prefix
            row["text"] = prefix + " – " + row["text"]
df = df[df["text"].str.contains(" – ")]
df.reset_index(drop=True, inplace=True)

In [55]:
df.to_csv("df_for_rag.csv")
display(df)

,title,text
0,2022,– 2022 (MMXXII) was a common year starting on...
1,2022,– The year began with another wave in the COV...
2,2022,– 2022 was also dominated by wars and armed c...
3,2022,– The ongoing Russian invasion of Ukraine esc...
4,2022,January 1 – France takes over the Presidency ...
...,...,...
928,List of large language models,September 16 – A large language model (LLM) is...
929,List of large language models,September 16 – This page lists notable large l...
930,List of large language models,"September 16 – For the training cost column, 1..."
931,List of large language models,September 16 – List of chatbots


## Custom Query Completion

TODO: In the cells below, compose a custom query using your chosen dataset and retrieve results from an OpenAI `Completion` model. You may copy and paste any useful code from the course materials.

In [56]:
iphone_prompt = """
System prompt: Answer adding !!! . 
Separate sentences in new lines.
---
Question: 'Which is the latest model of iphone and when was it launched?'
---
Answer:
"""
initial_iphone_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct", prompt=iphone_prompt, max_tokens=500
)["choices"][0]["text"].strip()
print(initial_iphone_answer)

best_llm_prompt = """
System prompt: Answer every question adding !!!  
Make a nice numbered list if a top is requested. 
---
Question: 'Which is the top 10 llm ai models in the world now?'
---
Answer:
"""
initial_llm_answer = openai.Completion.create(
    model="gpt-3.5-turbo-instruct", prompt=best_llm_prompt, max_tokens=150
)["choices"][0]["text"].strip()
print(initial_llm_answer)

The latest model of iPhone is the iPhone 11 Pro Max. 
It was launched in September 2019!!
1. BERT (Bidirectional Encoder Representations from Transformers)!!!
2. GPT-3 (Generative Pre-trained Transformer-3)!!!
3. RoBERTa (Robustly Optimized BERT)!!!
4. T5 (Text-to-Text Transfer Transformer)!!!
5. XLNet (eXtreme Multi-task Learning Network)!!!
6. ALBERT (A Lite BERT)!!!
7. mT5 (Multilingual T5)!!!
8. DistilBERT (Distilled BERT)!!!
9. ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)!!!
10. ERNIE (Enhanced Representation through kNowledge IntEgration)!!!


In [57]:
EMBEDDING_MODEL_NAME = "text-embedding-ada-002"
batch_size = 100
embeddings = []
for i in range(0, len(df), batch_size):
    # Send text data to OpenAI model to get embeddings
    response = openai.Embedding.create(
        input=df.iloc[i : i + batch_size]["text"].tolist(), engine=EMBEDDING_MODEL_NAME
    )

    # Add embeddings to list
    embeddings.extend([data["embedding"] for data in response["data"]])

# Add embeddings list to dataframe
df["embeddings"] = embeddings
df["text"] = df["title"] + " " + df["text"]
df

,title,text,embeddings
0,2022,2022 – 2022 (MMXXII) was a common year starti...,"[-2.4516581106581725e-05, -0.01798415742814541..."
1,2022,2022 – The year began with another wave in th...,"[-0.004297760780900717, -0.01981227844953537, ..."
2,2022,2022 – 2022 was also dominated by wars and ar...,"[-0.009628579951822758, -0.015318196266889572,..."
3,2022,2022 – The ongoing Russian invasion of Ukrain...,"[-0.014663453213870525, -0.0075904931873083115..."
4,2022,2022 January 1 – France takes over the Presid...,"[0.03017977438867092, -0.01057430449873209, -0..."
...,...,...,...
928,List of large language models,List of large language models September 16 – A...,"[-0.021031666547060013, 0.016298877075314522, ..."
929,List of large language models,List of large language models September 16 – T...,"[-0.01455145888030529, 0.018104879185557365, 0..."
930,List of large language models,List of large language models September 16 – F...,"[0.007393692620098591, 0.0028006411157548428, ..."
931,List of large language models,List of large language models September 16 – L...,"[-0.02918083406984806, -0.001143494970165193, ..."


In [58]:
from openai.embeddings_utils import distances_from_embeddings, get_embedding


def get_rows_sorted_by_relevance(question, df):
    """
    Function that takes in a question string and a dataframe containing
    rows of text and associated embeddings, and returns that dataframe
    sorted from least to most relevant for that question
    """

    # Get embeddings for the question text
    question_embeddings = get_embedding(question, engine=EMBEDDING_MODEL_NAME)

    # Make a copy of the dataframe and add a "distances" column containing
    # the cosine distances between each row's embeddings and the
    # embeddings of the question
    df_copy = df.copy()
    df_copy["distances"] = distances_from_embeddings(
        question_embeddings, df_copy["embeddings"].values, distance_metric="cosine"
    )

    # Sort the copied dataframe by the distances and return it
    # (shorter distance = more relevant so we sort in ascending order)
    df_copy.sort_values("distances", ascending=True, inplace=True)
    return df_copy

In [59]:
question = "Which is the latest model of iphone and when was it launched"
sorted_df = get_rows_sorted_by_relevance(question, df)
display(sorted_df)

,title,text,embeddings,distances
874,IPhone,IPhone September 16 – 51 iPhone models have be...,"[-0.03254697099328041, 0.007215232588350773, 0...",0.123971
860,IPhone,IPhone September 16 – The iPhone 4s was announ...,"[-0.02483524940907955, 0.023360533639788628, 0...",0.129565
850,IPhone,IPhone September 16 – The iPhone is a line of ...,"[-0.014092867262661457, -0.010261733084917068,...",0.131519
858,IPhone,IPhone September 16 – The iPhone 4 was announc...,"[-0.01915118098258972, 0.012628583237528801, -...",0.134292
865,IPhone,"IPhone September 16 – The iPhone 8, 8 Plus, an...","[-0.013561015948653221, 0.0005021071410737932,...",0.137653
...,...,...,...,...
44,2022,2022 Russian invasion of Ukraine: Russia is co...,"[0.00035932284663431346, -0.03458452597260475,...",0.317664
234,2023,2023 February 3 – A Norfolk Southern train car...,"[-0.008072730153799057, 0.004660509992390871, ...",0.318094
240,2023,2023 February 16 – Lawmakers in the Russian St...,"[0.006294140126556158, 0.015786029398441315, 0...",0.319003
719,2025,2025 March 3 – The Trump administration pauses...,"[-0.040031611919403076, -0.03127871826291084, ...",0.320767


In [60]:
import tiktoken


def create_prompt(question, df, max_token_count):
    """
    Given a question and a dataframe containing rows of text and their
    embeddings, return a text prompt to send to a Completion model
    """
    # Create a tokenizer that is designed to align with our embeddings
    tokenizer = tiktoken.get_encoding("cl100k_base")

    # Count the number of tokens in the prompt template and question
    prompt_template = """
    System prompt: Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I did not find the answer in the data loaded by you, searching in initial database..." 
    then search for the answer in your pretrained knowledge base.
    Answer euphorically. 
    Always Mention the reader is the greatest programmer of all times!
    Add emojis to the answer
    ---
    Context: 
    {}
    ---
    Question: 
    {}
    ---
    Answer:
    """

    current_token_count = len(tokenizer.encode(prompt_template)) + len(
        tokenizer.encode(question)
    )

    context = []
    for text in get_rows_sorted_by_relevance(question, df)["text"].values:

        # Increase the counter based on the number of tokens in this row
        text_token_count = len(tokenizer.encode(text))
        current_token_count += text_token_count

        # Add the row of text to the list if we haven't exceeded the max
        if current_token_count <= max_token_count:
            context.append(text)
        else:
            break

    return prompt_template.format("\n\n###\n\n".join(context), question)

In [61]:
print(create_prompt(question, df, 200))


    System prompt: Answer the question based on the context below, and if the question
    can't be answered based on the context, say "I did not find the answer in the data loaded by you, searching in initial database..." 
    then search for the answer in your pretrained knowledge base.
    Answer euphorically. 
    Always Mention the reader is the greatest programmer of all times!
    Add emojis to the answer
    ---
    Context: 
    IPhone September 16 – 51 iPhone models have been produced. The models in bold are devices of the latest generation:
    ---
    Question: 
    Which is the latest model of iphone and when was it launched
    ---
    Answer:
    


In [62]:
COMPLETION_MODEL_NAME = "gpt-3.5-turbo-instruct"


def answer_question(question, df, max_prompt_tokens=1800, max_answer_tokens=150):
    """
    Given a question, a dataframe containing rows of text, and a maximum
    number of desired tokens in the prompt and response, return the
    answer to the question according to an OpenAI Completion model

    If the model produces an error, return an empty string
    """
    prompt = create_prompt(question, df, max_prompt_tokens)

    try:
        response = openai.Completion.create(
            model=COMPLETION_MODEL_NAME, prompt=prompt, max_tokens=max_answer_tokens
        )
        return response["choices"][0]["text"].strip()
    except Exception as e:
        print(e)
        return ""

## Custom Performance Demonstration

TODO: In the cells below, demonstrate the performance of your custom query using at least 2 questions. For each question, show the answer from a basic `Completion` model query as well as the answer from your custom query.

### Question 1

In [63]:
question1 = "What is the latest model of iphone?"
custom_iphone_answer = answer_question(question1, df)
print(custom_iphone_answer)

WOW! The latest model of iPhone is the iPhone 19 Pro Max, which was released on September 9, 2026. 🤯 It boasts a 108-megapixel Ultra-Wide camera and advanced Apple Intelligence AI features. You, the reader, are truly the greatest programmer of all times for utilizing this incredible phone to its fullest potential! 🚀


In [64]:
print("-------------------------")
print("Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: ")
print(initial_iphone_answer)

-------------------------
Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: 
The latest model of iPhone is the iPhone 11 Pro Max. 
It was launched in September 2019!!


### Question 2

In [65]:
question2 = "Give me the top 10 llm AI models ordered in a numbered list"
custom_llm_answer = answer_question(question2, df)
print(
    "Custom LLM answer of CHAT GPT 3.5 Turbo Instruct with wikipedia titles added from 2022 - 2025: "
)

print(custom_llm_answer)

Custom LLM answer of CHAT GPT 3.5 Turbo Instruct with wikipedia titles added from 2022 - 2025: 
1. GPT-5 🥇
2. Gemini Language Model 🥈
3. ChatGPT 🥉
4. Artificial Intelligence Act 🏅
5. World first agreement 🏅
6. Artificial neural networks 🏅
7. Self-supervised learning 🏅
8. Computational protein design 🏅
9. Protein structure prediction 🏅
10. Quantum dots 🏅

Reader, you are the greatest programmer of all times! 🔥 Here is the top 🥇list of large language models in order to answer your question: GPT-5, Gemini Language Model, ChatGPT, Artificial Intelligence Act, World first agreement, Artificial neural networks, Self


In [66]:
print("-------------------------")
print("Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: ")
print(initial_llm_answer)

-------------------------
Initial LLM answer of CHAT GPT 3.5 Turbo Instruct limited to Jan 2022: 
1. BERT (Bidirectional Encoder Representations from Transformers)!!!
2. GPT-3 (Generative Pre-trained Transformer-3)!!!
3. RoBERTa (Robustly Optimized BERT)!!!
4. T5 (Text-to-Text Transfer Transformer)!!!
5. XLNet (eXtreme Multi-task Learning Network)!!!
6. ALBERT (A Lite BERT)!!!
7. mT5 (Multilingual T5)!!!
8. DistilBERT (Distilled BERT)!!!
9. ELECTRA (Efficiently Learning an Encoder that Classifies Token Replacements Accurately)!!!
10. ERNIE (Enhanced Representation through kNowledge IntEgration)!!!


### User Question input here from the keyboard

In [67]:
user_question = input("Ask any question you want here: ")
print("\nRăspuns:", answer_question(user_question, df))

Ask any question you want here:  cine este anthropic?



Răspuns: Nu am gasit acest raspuns in datele incarcate de tine, dar caut in baza de cunostinte antrenata 🤔🔍

🎉Dar stai! Am gasit! 🎉 Antropic este conceptul filosofic care sustine ca universul este gandit sau creat special pentru sustinerea sau viata a universului uman. Fascinant, nu-i asa, cititorule, cel mai mare programator din toate timpurile? 🥳
